In [3]:
# default_exp auth

# auth


> API details.

In [4]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [43]:
# export

import json
import os

import stravalib
import webbrowser
from fastcore.all import *


In [125]:
# export
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
client_id, secret = os.getenv("client_id"), os.getenv("secret")

In [124]:
f = "auth_token.json"
find_dotenv(f,raise_error_if_not_found=True)

'/home/tako/dev/strava_overview/auth_token.json'

In [121]:
# export
class AuthToken:
    def __init__(self, f="auth_token.json"):
        try:
            f = find_dotenv(f,raise_error_if_not_found=True)
        except:
            f = "auth_token.json"
        self.path = f

    def exists(self):
        return os.path.isfile(self.path)

    def set(self, dct: {}):
        json.dump(dct, open(self.path, "w"))

    def get(self, state=None):
        if state is not None:
            return state
        return json.load(open(self.path, "r"))

def save_code_to_token(state, auth=None, client = None):
    client = ifnone(client, stravalib.client.Client())
    auth = ifnone(auth, AuthToken())
    token = client.exchange_code_for_token(
            client_id=client_id, client_secret=secret, code=state["code"],
        )
    state = token
    auth.set(state)
    

def get_access_token(state: {} = None):
    client = stravalib.client.Client()
    auth = AuthToken()
    if not auth.exists():
        host = os.getenv("host")
        url = 'http://' +host+ ':5555/authorized'
        authorize_url = client.authorization_url(
            client_id=client_id, redirect_uri=url, scope="activity:read_all"
        )
        webbrowser.open(authorize_url)
    else:
        state = auth.get(state)
        if "refresh_token" in state:
            token = client.refresh_access_token(
                client_id=client_id,
                client_secret=secret,
                refresh_token=state["refresh_token"],
            )
        else:
            token = client.exchange_code_for_token(
                client_id=client_id, client_secret=secret, code=state["code"],
            )
        state["refresh_token"] = token["refresh_token"]
        state["access_token"] = token["access_token"]
        auth.set(state)
        return state["access_token"]


In [122]:
get_access_token()

'4b91ea8c84b147966f73aa48457308a83d693141'

In [123]:
from nbdev.export import *
notebook2script()


Converted 00_auth.ipynb.
Converted 01_to_csv.ipynb.
Converted 02_parse.ipynb.
Converted index.ipynb.
